# APS 4

Funções padrões

In [2]:
from funcoesTermosol import plota, importa, geraSaida

In [3]:
import math
import numpy as np

A funcao 'importa' retorna o numero de nos [nn], a matriz dos nos [N], o numero
de membros [nm], a matriz de incidencia [Inc], o numero de cargas [nc], o vetor
carregamento [F], o numero de restricoes [nr] e o vetor de restricoes [R] 
contidos no arquivo de entrada.

In [4]:
[nn,N,nm,Inc,nc,F,nr,R] = importa('entrada.xls')

In [5]:
print([nn,nm,nc,nr])

[3, 3, 2, 3]


In [6]:
Inc

array([[1.0e+00, 2.0e+00, 2.1e+11, 2.0e-04],
       [2.0e+00, 3.0e+00, 2.1e+11, 2.0e-04],
       [3.0e+00, 1.0e+00, 2.1e+11, 2.0e-04]])

In [7]:
R

array([[0.],
       [2.],
       [3.]])

In [8]:
F

array([[   0.],
       [   0.],
       [   0.],
       [   0.],
       [ 150.],
       [-100.]])

In [9]:
N

array([[0. , 0. , 0.3],
       [0. , 0.4, 0.4]])

In [10]:
N.shape

(2, 3)

### Função para calcular distâncias, cossenos e senos

In [11]:
def calc_dist_cs(N, Inc):
    l_list = []
    c_list = []
    s_list = []
    for i in range(N.shape[1]):
        if i == N.shape[1]-1:
            l = (math.sqrt((N[0][i] - N[0][0])**2 + (N[1][i] - N[1][0])**2))
            c = (N[0][0]-N[0][i])/l
            s = (N[1][0]-N[1][i])/l
            l_list.append(l)
            c_list.append(c)
            s_list.append(s)
        else:
            l = (math.sqrt((N[0][i] - N[0][i+1])**2 + (N[1][i] - N[1][i+1])**2))
            c = (N[0][i+1]-N[0][i])/l
            s = (N[1][i+1]-N[1][i])/l
            l_list.append(l)
            c_list.append(c)
            s_list.append(s)
    l_array = np.c_[l_list]
    c_array = np.c_[c_list]
    s_array = np.c_[s_list]
    Inc = np.hstack((Inc, l_array))
    Inc = np.hstack((Inc, c_array))
    Inc = np.hstack((Inc, s_array))
    return Inc

ex:

In [12]:
Inc = calc_dist_cs(N,Inc)
Inc

array([[ 1.0e+00,  2.0e+00,  2.1e+11,  2.0e-04,  4.0e-01,  0.0e+00,
         1.0e+00],
       [ 2.0e+00,  3.0e+00,  2.1e+11,  2.0e-04,  3.0e-01,  1.0e+00,
         0.0e+00],
       [ 3.0e+00,  1.0e+00,  2.1e+11,  2.0e-04,  5.0e-01, -6.0e-01,
        -8.0e-01]])

### Função para achar matrizes de rigidez

In [13]:
def matrizes_rigidez(Inc, nm):
    matrizes = np.zeros(shape=(nm,4,4))   # Considerando a treliça com duas extremidades
    matrizes_coordenada = np.zeros(shape=(nm,4,4,2))   # Considerando as coordenadas alinhadas com os graus de liberdade
    
    i = 0
    for cs in Inc:
        rigidez = cs[2]*cs[3]/cs[4]  # E*A/L para todos os membros
        
        c = cs[5]
        s = cs[6]
        matriz_cs = [[c**2 ,     c*s,   -(c**2), -(c*s) ],
                     [c*s ,      s**2,  -(c*s),  -(s**2)],
                     [-(c**2),  -(c*s),  c**2,    c *s  ],
                     [-(c*s),   -(s**2), c*s,     s**2  ]]
       


        matriz_coord = np.zeros(shape=(4,4,2))

        grau_linha = 0
        grau_coluna = 0
        for m in range(4):
            if   m == 0:
                grau_linha = cs[0]*2-1
            elif m == 1:
                grau_linha = cs[0]*2 
            elif m == 2:
                grau_linha = cs[1]*2-1 
            elif m == 3:
                grau_linha = cs[1]*2 
                
            for n in range(4):
                if   n == 0:
                    grau_coluna = cs[0]*2-1
                elif n == 1:
                    grau_coluna = cs[0]*2
                elif n == 2:
                    grau_coluna = cs[1]*2-1
                elif n == 3:
                    grau_coluna = cs[1]*2

                matriz_coord[m][n] = (grau_linha-1, grau_coluna-1)
       
        
        matriz_cs = np.multiply(matriz_cs, rigidez),
        
        matrizes_coordenada[i,:,:,:] = matriz_coord[:,:,:]
        matrizes[i,:,:] = np.add(matrizes[i,:,:], matriz_cs)

        i+=1
       
    return matrizes, matrizes_coordenada


ex

In [14]:
a,b= matrizes_rigidez(Inc,nm)
a

array([[[ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
        [ 0.000e+00,  1.050e+08,  0.000e+00, -1.050e+08],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
        [ 0.000e+00, -1.050e+08,  0.000e+00,  1.050e+08]],

       [[ 1.400e+08,  0.000e+00, -1.400e+08,  0.000e+00],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
        [-1.400e+08,  0.000e+00,  1.400e+08,  0.000e+00],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00]],

       [[ 3.024e+07,  4.032e+07, -3.024e+07, -4.032e+07],
        [ 4.032e+07,  5.376e+07, -4.032e+07, -5.376e+07],
        [-3.024e+07, -4.032e+07,  3.024e+07,  4.032e+07],
        [-4.032e+07, -5.376e+07,  4.032e+07,  5.376e+07]]])

In [15]:
b

array([[[[0., 0.],
         [0., 1.],
         [0., 2.],
         [0., 3.]],

        [[1., 0.],
         [1., 1.],
         [1., 2.],
         [1., 3.]],

        [[2., 0.],
         [2., 1.],
         [2., 2.],
         [2., 3.]],

        [[3., 0.],
         [3., 1.],
         [3., 2.],
         [3., 3.]]],


       [[[2., 2.],
         [2., 3.],
         [2., 4.],
         [2., 5.]],

        [[3., 2.],
         [3., 3.],
         [3., 4.],
         [3., 5.]],

        [[4., 2.],
         [4., 3.],
         [4., 4.],
         [4., 5.]],

        [[5., 2.],
         [5., 3.],
         [5., 4.],
         [5., 5.]]],


       [[[4., 4.],
         [4., 5.],
         [4., 0.],
         [4., 1.]],

        [[5., 4.],
         [5., 5.],
         [5., 0.],
         [5., 1.]],

        [[0., 4.],
         [0., 5.],
         [0., 0.],
         [0., 1.]],

        [[1., 4.],
         [1., 5.],
         [1., 0.],
         [1., 1.]]]])

### Função para matriz de rigidez global

In [45]:
def rigidez_global(matrizes_rigidez, matrizes_coordenadas):
    matriz_global = np.zeros(shape=(nn*2,nn*2))   # Considerando 2 graus de liberdade~
    elemento = 0 
    pos_lin = 0
    pos_col = 0
    for matriz_coordenada in matrizes_coordenadas:
        for linha in matriz_coordenada:
            for coordenada in linha:
                matriz_global[int(coordenada[0])][int(coordenada[1])] += matrizes_rigidez[elemento][pos_lin][pos_col]
                pos_col += 1
            pos_col = 0
            pos_lin += 1
        pos_lin = 0
        elemento += 1
        

    return matriz_global


ex

In [46]:
mat_global = rigidez_global(a, b)
mat_global

array([[ 3.0240e+07,  4.0320e+07,  0.0000e+00,  0.0000e+00, -3.0240e+07,
        -4.0320e+07],
       [ 4.0320e+07,  1.5876e+08,  0.0000e+00, -1.0500e+08, -4.0320e+07,
        -5.3760e+07],
       [ 0.0000e+00,  0.0000e+00,  1.4000e+08,  0.0000e+00, -1.4000e+08,
         0.0000e+00],
       [ 0.0000e+00, -1.0500e+08,  0.0000e+00,  1.0500e+08,  0.0000e+00,
         0.0000e+00],
       [-3.0240e+07, -4.0320e+07, -1.4000e+08,  0.0000e+00,  1.7024e+08,
         4.0320e+07],
       [-4.0320e+07, -5.3760e+07,  0.0000e+00,  0.0000e+00,  4.0320e+07,
         5.3760e+07]])

A funcao 'geraSaida' cria um arquivo nome.txt contendo as reacoes de apoio Ft, 
deslocamentos Ut, deformacoes Epsi, forcas Fi e tensoes Ti internas. 
As entradas devem ser vetores coluna.